In [5]:
%load_ext autoreload
%autoreload 2

import os.path as osp

import torch

from torch_geometric.nn import SchNet
from torch_geometric.data import DataLoader
from torch_geometric.datasets import QM9

path = '../datasets/qm9_geometric_schnet/'
dataset = QM9(path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


target_dict = {0: 'mu, D, Dipole moment', 
               1: 'alpha, {a_0}^3, Isotropic polarizability', 
               2: 'epsilon_{HOMO}, eV, Highest occupied molecular orbital energy',
               3: 'epsilon_{LUMO}, eV, Lowest unoccupied molecular orbital energy',
               4: 'Delta, eV, Gap between HOMO and LUMO',
               5: '< R^2 >, {a_0}^2, Electronic spatial extent',
               6: 'ZPVE, eV, Zero point vibrational energy', 
               7: 'U_0, eV, Internal energy at 0K',
               8: 'U, eV, Internal energy at 298.15K', 
               9: 'H, eV, Enthalpy at 298.15K',
               10: 'G, eV, Free energy at 298.15K',  
               11: 'c_{v}, cal\(mol K), Heat capacity at 298.15K'}

for target in range(12):
    model, datasets = SchNet.from_qm9_pretrained(path, dataset, target)
    train_dataset, val_dataset, test_dataset = datasets

    model = model.to(device)
    loader = DataLoader(test_dataset, batch_size=256)

    maes = []
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            pred = model(data.z, data.pos, data.batch)
        mae = (pred.view(-1) - data.y[:, target]).abs()
        maes.append(mae)

    mae = torch.cat(maes, dim=0)

    # Report meV instead of eV.
    mae = 1000 * mae if target in [2, 3, 4, 6, 7, 8, 9, 10] else mae

    print(f'Target: {target:02d}, MAE: {mae.mean():.5f} ± {mae.std():.5f}, {target_dict[target]}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Target: 00, MAE: 0.02079 ± 0.03071, mu, D, Dipole moment
Target: 01, MAE: 0.12105 ± 0.39941, alpha, {a_0}^3, Isotropic polarizability
Target: 02, MAE: 46.56833 ± 48.21923, epsilon_{HOMO}, eV, Highest occupied molecular orbital energy
Target: 03, MAE: 38.17754 ± 51.09161, epsilon_{LUMO}, eV, Lowest unoccupied molecular orbital energy
Target: 04, MAE: 73.87022 ± 78.23814, Delta, eV, Gap between HOMO and LUMO
Target: 05, MAE: 0.15656 ± 0.34679, < R^2 >, {a_0}^2, Electronic spatial extent
Target: 06, MAE: 1.59998 ± 1.64683, ZPVE, eV, Zero point vibrational energy
Target: 07, MAE: 11.99382 ± 31.97339, U_0, eV, Internal energy at 0K
Target: 08, MAE: 11.94271 ± 22.93234, U, eV, Internal energy at 298.15K
Target: 09, MAE: 12.03425 ± 22.43000, H, eV, Enthalpy at 298.15K
Target: 10, MAE: 12.88869 ± 25.78860, G, eV, Free energy at 298.15K
Target: 11, MAE: 0.03342 ± 0.04913, c_{v}, cal\(mol K), Heat capacity at